In [7]:
import requests
import pandas as pd
import io
import os
import mysql.connector

In [8]:
import sqlalchemy

ModuleNotFoundError: No module named 'sqlalchemy'

In [9]:
import hashlib

In [10]:
earthquake_url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.csv'

In [11]:
s=requests.get(earthquake_url).content

In [10]:
s

b'time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource\n2023-09-19T18:58:20.288Z,59.608,-154.0458,167.9,2.6,ml,,,,0.68,ak,ak023c1lv7d8,2023-09-19T19:01:45.875Z,"Southern Alaska",earthquake,,0.9,,,automatic,ak,ak\n2023-09-19T18:06:06.117Z,-24.3722,-67.1545,167.441,4.3,mwr,81,55,1.697,1.06,us,us7000kwqc,2023-09-19T19:47:54.040Z,"86 km W of San Antonio de los Cobres, Argentina",earthquake,6.67,4.837,,,reviewed,us,guc\n2023-09-19T16:52:59.794Z,54.3662,-160.8867,35,4.4,mwr,56,161,1.013,0.89,us,us7000kwpy,2023-09-19T17:25:25.521Z,"110 km SSW of Sand Point, Alaska",earthquake,4.96,1.926,0.083,14,reviewed,us,us\n2023-09-19T16:04:10.950Z,51.5096,16.1694,5,4.6,mb,102,44,1.165,0.9,us,us7000kwpp,2023-09-19T18:45:42.510Z,"6 km W of Rudna, Poland",earthquake,4.26,1.877,0.066,68,reviewed,us,us\n2023-09-19T15:58:41.750Z,40.5628,-125.6305,10,2.7,ml,14,276,1.244,0.38,us,us7000kwpn,2023-09-19T17:24:

In [12]:
earthquake_df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [14]:
earthquake_df.dtypes

time                object
latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place               object
type                object
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status              object
locationSource      object
magSource           object
dtype: object

In [13]:
import pandas as pd
import numpy as np

def pandas_to_mysql_type(pandas_type):
    """
    Convert a Pandas data type to its MySQL equivalent.
    
    Args:
        pandas_type (str): The Pandas data type as a string.

    Returns:
        str: The MySQL data type as a string.
    """
    if pandas_type == 'object':
        return 'TEXT'
    elif pandas_type == 'int64':
        return 'INT'
    elif pandas_type == 'float64':
        return 'FLOAT'
    elif pandas_type == 'datetime64':
        return 'DATETIME'
    elif pandas_type == 'bool':
        return 'BOOL'
    else:
        # Handle other cases or raise an error for unsupported types.
        raise ValueError(f"Unsupported Pandas type: {pandas_type}")

def convert_pandas_to_mysql_types(dataframe):
    """
    Convert Pandas column types to MySQL data types for a DataFrame.

    Args:
        dataframe (pd.DataFrame): The Pandas DataFrame to convert.

    Returns:
        dict: A dictionary mapping column names to MySQL data types.
    """
    mysql_types = {}
    for column_name, dtype in dataframe.dtypes.items():
        mysql_type = pandas_to_mysql_type(dtype.name)
        mysql_types[column_name] = mysql_type
    return mysql_types

# Example usage:
data = {'int_col': [1, 2, 3],
        'str_col': ['A', 'B', 'C'],
        'float_col': [1.1, 2.2, 3.3],
        'date_col': pd.to_datetime(['2021-01-01', '2021-02-01', '2021-03-01']),
        'bool_col': [True, False, True]}

df = pd.DataFrame(data)

mysql_column_types = convert_pandas_to_mysql_types(df)
print(mysql_column_types)


ValueError: Unsupported Pandas type: datetime64[ns]

In [18]:
mysql_column_types = convert_pandas_to_mysql_types(earthquake_df)

In [16]:
converted_df

{'time': 'TEXT',
 'latitude': 'FLOAT',
 'longitude': 'FLOAT',
 'depth': 'FLOAT',
 'mag': 'FLOAT',
 'magType': 'TEXT',
 'nst': 'FLOAT',
 'gap': 'FLOAT',
 'dmin': 'FLOAT',
 'rms': 'FLOAT',
 'net': 'TEXT',
 'id': 'TEXT',
 'updated': 'TEXT',
 'place': 'TEXT',
 'type': 'TEXT',
 'horizontalError': 'FLOAT',
 'depthError': 'FLOAT',
 'magError': 'FLOAT',
 'magNst': 'FLOAT',
 'status': 'TEXT',
 'locationSource': 'TEXT',
 'magSource': 'TEXT'}

In [6]:
earthquake_df.dtypes

time                object
latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place               object
type                object
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status              object
locationSource      object
magSource           object
dtype: object

In [21]:
table_name = 'portfolio.earthquake_table'
create_table_sql = f"CREATE TABLE {table_name} ("

for column_name, mysql_type in mysql_column_types.items():
    create_table_sql += f"{column_name} {mysql_type}, "

# Remove the trailing comma and space
create_table_sql = create_table_sql[:-2]

create_table_sql += ");"

In [22]:
create_table_sql

'CREATE TABLE portfolio.earthquake_table (time TEXT, latitude FLOAT, longitude FLOAT, depth FLOAT, mag FLOAT, magType TEXT, nst FLOAT, gap FLOAT, dmin FLOAT, rms FLOAT, net TEXT, id TEXT, updated TEXT, place TEXT, type TEXT, horizontalError FLOAT, depthError FLOAT, magError FLOAT, magNst FLOAT, status TEXT, locationSource TEXT, magSource TEXT);'

In [46]:
connection = mysql.connector.connect(
        host=os.environ["MYSQL_ENDPOINT"],
        user=os.environ["MYSQL_USER"],
        password=os.environ["MYSQL_PASS"]
    )


In [47]:
cursor = connection.cursor()

In [25]:
cursor.execute(create_table_sql)

In [26]:
cursor.close()

True

In [28]:
columns = ', '.join(mysql_column_types.keys())
placeholders = ', '.join(['%s'] * len(mysql_column_types))
insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"


In [42]:
insert_sql

'INSERT INTO portfolio.earthquake_table (time, latitude, longitude, depth, mag, magType, nst, gap, dmin, rms, net, id, updated, place, type, horizontalError, depthError, magError, magNst, status, locationSource, magSource) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [36]:
earthquake_df = earthquake_df.fillna(0.00)

In [33]:
mysql_column_types

{'time': 'TEXT',
 'latitude': 'FLOAT',
 'longitude': 'FLOAT',
 'depth': 'FLOAT',
 'mag': 'FLOAT',
 'magType': 'TEXT',
 'nst': 'FLOAT',
 'gap': 'FLOAT',
 'dmin': 'FLOAT',
 'rms': 'FLOAT',
 'net': 'TEXT',
 'id': 'TEXT',
 'updated': 'TEXT',
 'place': 'TEXT',
 'type': 'TEXT',
 'horizontalError': 'FLOAT',
 'depthError': 'FLOAT',
 'magError': 'FLOAT',
 'magNst': 'FLOAT',
 'status': 'TEXT',
 'locationSource': 'TEXT',
 'magSource': 'TEXT'}

In [34]:
earthquake_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-09-25T18:10:11.669Z,34.070333,-97.432333,6.3100,2.51,ml,77.0,46.0,0.00000,0.37,...,2023-09-25T18:25:07.040Z,"10 km S of Wilson, Oklahoma",earthquake,NaN,0.900000,0.240000,22.0,reviewed,ok,ok
1,2023-09-25T17:03:19.290Z,17.988500,-66.854167,11.5300,2.50,md,4.0,164.0,0.02737,0.18,...,2023-09-25T17:19:51.500Z,"3 km SE of Palomas, Puerto Rico",earthquake,4.720000,2.140000,0.241869,3.0,reviewed,pr,pr
2,2023-09-25T16:42:01.180Z,17.954833,-66.843833,12.0600,2.47,md,7.0,207.0,0.03926,0.11,...,2023-09-25T16:57:02.100Z,"5 km SSW of Indios, Puerto Rico",earthquake,0.750000,0.510000,0.180713,5.0,reviewed,pr,pr
3,2023-09-25T16:00:12.160Z,19.332500,-155.139664,-0.6600,2.77,ml,47.0,163.0,NaN,0.18,...,2023-09-25T16:59:09.040Z,"14 km S of Fern Forest, Hawaii",earthquake,0.490000,0.170000,3.130000,30.0,automatic,hv,hv
4,2023-09-25T15:48:49.340Z,-6.950000,130.002900,90.6210,4.70,mb,51.0,87.0,2.41400,0.67,...,2023-09-25T16:28:21.040Z,Banda Sea,earthquake,9.020000,2.358000,0.089000,38.0,reviewed,us,us
5,2023-09-25T15:21:09.820Z,18.002667,-66.857833,9.2900,2.54,md,5.0,145.0,0.03421,0.10,...,2023-09-25T15:41:28.260Z,"2 km SE of Palomas, Puerto Rico",earthquake,3.030000,1.900000,0.127270,2.0,reviewed,pr,pr
6,2023-09-25T13:31:51.937Z,47.735800,-128.443400,10.0000,4.10,mb,57.0,182.0,2.31600,0.85,...,2023-09-25T20:10:14.536Z,off the coast of Washington,earthquake,9.090000,1.988000,0.076000,47.0,reviewed,us,us
7,2023-09-25T12:11:59.322Z,-6.715700,129.737900,168.6760,4.50,mb,37.0,97.0,2.18500,0.75,...,2023-09-25T12:27:37.040Z,Banda Sea,earthquake,8.680000,7.830000,0.106000,26.0,reviewed,us,us
8,2023-09-25T11:02:21.280Z,17.878667,-66.972167,8.5100,2.46,md,4.0,253.0,0.11340,0.17,...,2023-09-25T11:38:42.660Z,"12 km SSW of Guánica, Puerto Rico",earthquake,1.940000,4.710000,0.037029,2.0,reviewed,pr,pr
9,2023-09-25T09:10:06.640Z,36.683334,-121.352669,3.8100,2.61,md,47.0,54.0,0.02458,0.19,...,2023-09-25T16:51:12.426Z,"12 km SSW of Tres Pinos, CA",earthquake,0.270000,0.970000,0.150000,55.0,automatic,nc,nc


In [48]:
#
# insert data into table
for index, row in earthquake_df.iterrows():
    values = [row[column] for column in mysql_column_types.keys()]
    print(values)
    cursor.execute(insert_sql, values)

['2023-09-25T18:10:11.669Z', 34.07033333, -97.43233333, 6.31, 2.51, 'ml', 77.0, 46.0, 0.0, 0.37, 'ok', 'ok2023svfh', '2023-09-25T18:25:07.040Z', '10 km S of Wilson, Oklahoma', 'earthquake', 0.0, 0.9, 0.24, 22.0, 'reviewed', 'ok', 'ok']
['2023-09-25T17:03:19.290Z', 17.9885, -66.8541666666667, 11.53, 2.5, 'md', 4.0, 164.0, 0.02737, 0.18, 'pr', 'pr71425878', '2023-09-25T17:19:51.500Z', '3 km SE of Palomas, Puerto Rico', 'earthquake', 4.72, 2.14, 0.241869394901518, 3.0, 'reviewed', 'pr', 'pr']
['2023-09-25T16:42:01.180Z', 17.9548333333333, -66.8438333333333, 12.06, 2.47, 'md', 7.0, 207.0, 0.03926, 0.11, 'pr', 'pr71425868', '2023-09-25T16:57:02.100Z', '5 km SSW of Indios, Puerto Rico', 'earthquake', 0.75, 0.51, 0.180713256236441, 5.0, 'reviewed', 'pr', 'pr']
['2023-09-25T16:00:12.160Z', 19.3325004577637, -155.139663696289, -0.660000026226044, 2.77, 'ml', 47.0, 163.0, 0.0, 0.180000007, 'hv', 'hv73585172', '2023-09-25T16:59:09.040Z', '14 km S of Fern Forest, Hawaii', 'earthquake', 0.49, 0.170

In [50]:
connection.commit()

In [51]:
cursor.close()

True

In [13]:
def create_hash_column(dataframe):
    """
    Create a hash column by combining all other columns in the DataFrame into a string
    and hashing it using SHA-256.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame.

    Returns:
        pandas.DataFrame: The DataFrame with an additional 'hash_column' containing the hash values.
    """
    # Combine all columns into a single string
    combined_string = dataframe.apply(lambda row: ''.join(map(str, row)), axis=1)

    # Create a new column with the hash values
    dataframe['hash_column'] = combined_string.apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

    return dataframe

In [14]:
df = create_hash_column(earthquake_df)

In [ ]:
connection.cursor

In [15]:
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,hash_column
0,2023-09-20T06:15:49.656Z,59.950500,-152.755800,82.600000,3.20,ml,NaN,NaN,NaN,0.640000,...,"55 km WNW of Anchor Point, Alaska",earthquake,NaN,0.800000,NaN,NaN,automatic,ak,ak,0efa38e4cf01e8daa77444edc26764c4ca7fa544a57261...
1,2023-09-20T05:57:31.080Z,18.147833,-67.396667,9.980000,2.70,md,7.0,228.00000,0.248000,0.290000,...,Mona Passage,earthquake,1.480000,0.970000,0.029072,4.0,reviewed,pr,pr,03404e87fbb3302884e633964b03153fe338a788d09caf...
2,2023-09-20T05:42:41.464Z,-20.719600,-178.340000,548.944000,4.60,mb,77.0,38.00000,2.966000,0.650000,...,Fiji region,earthquake,11.130000,7.316000,0.037000,218.0,reviewed,us,us,9e04b79add22265f82fd71dd1a8f54c107378563845437...
3,2023-09-20T05:21:37.750Z,17.962333,-66.825333,13.760000,2.46,md,10.0,204.00000,0.079910,0.120000,...,"3 km S of Indios, Puerto Rico",earthquake,0.780000,0.450000,0.122342,5.0,reviewed,pr,pr,96e8642d7480968f932952f2c3483f9ebd46863e25cdc7...
4,2023-09-20T04:32:39.430Z,19.119500,-65.592800,36.000000,3.50,md,12.0,268.00000,0.947500,0.420000,...,"80 km NNE of Vieques, Puerto Rico",earthquake,3.760000,18.270000,0.110000,9.0,reviewed,pr,pr,9e44851530ad772e89fa4806e3b0ab425f7180af72cc4f...
5,2023-09-20T03:01:50.890Z,68.187700,-145.567400,0.000000,2.70,ml,NaN,NaN,NaN,0.780000,...,"6 km N of Arctic Village, Alaska",earthquake,NaN,0.500000,NaN,NaN,automatic,ak,ak,efa299b8fe253f33cd948b0b0467c8fd88cc7d099a2778...
6,2023-09-20T02:39:40.680Z,19.052500,-155.387665,32.320000,2.55,ml,50.0,205.00000,NaN,0.110000,...,"19 km SSE of Pāhala, Hawaii",earthquake,0.610000,0.740000,2.760000,19.0,automatic,hv,hv,1f78677a09428010231ab701171abb5a4d23d3fc900224...
7,2023-09-20T02:39:38.398Z,38.184300,-117.779600,9.100000,3.20,ml,7.0,52.09000,0.121000,0.155000,...,Nevada,earthquake,NaN,2.400000,0.160000,4.0,reviewed,nn,nn,0974907f5b7e779f1839e22e1cc54159d970f8ca99d6cc...
8,2023-09-20T02:17:19.149Z,62.065200,-149.767600,32.600000,2.50,ml,NaN,NaN,NaN,0.460000,...,"11 km SSE of Susitna North, Alaska",earthquake,NaN,0.400000,NaN,NaN,automatic,ak,ak,b8ad696fdc0fda36329efa0bbc65e506382b1517fc0867...
9,2023-09-20T01:29:58.715Z,32.428894,-102.138553,6.529419,2.90,ml,27.0,271.00000,0.042045,0.300000,...,"38 km SSW of Los Ybanez, Texas",earthquake,1.668848,1.200343,0.200000,36.0,reviewed,tx,tx,315b56b403d42340890b3dd7538950f52d9a4968b073cf...


In [3]:
os.getcwd()


'/Users/nickphom/PortfolioWebsite/Nicholasphom.github.io/PortfolioCode/Project1'

In [ ]:
os.se

In [2]:
os.environ["MYSQL_PASS"]

'e0zPJ6XzCLHIago4rRKc'

In [5]:
print(os.getenv("MYSQL_PASS"))

None
